In [1]:
%env WANDB_NOTEBOOK_NAME Train 1 head mixer.ipynb
%env CUDA_DEVICE_ORDER PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES 0
%env PYTORCH_CUDA_ALLOC_CONF backend:cudaMallocAsync

env: WANDB_NOTEBOOK_NAME=Train 1 head mixer.ipynb
env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0
env: PYTORCH_CUDA_ALLOC_CONF=backend:cudaMallocAsync


In [2]:
import torch
# UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
torch.set_float32_matmul_precision('high')

In [3]:
from mp_20_utils import load_all_data
device = 'cuda'
dataset = 'mp_20_biternary'
datasets_pd, torch_datasets, site_to_ids, element_to_ids, spacegroup_to_ids, max_len, max_enumeration, enumeration_stop, enumeration_pad = load_all_data(
    dataset=dataset)
print(max_len, max_enumeration, enumeration_stop, enumeration_pad)

20 7 8 9


In [4]:
from cascade_transformer.model import CascadeTransformer
from wyckoff_transformer import WyckoffTrainer
from tokenization import PAD_TOKEN, MASK_TOKEN
n_space_groups = len(spacegroup_to_ids)
# Not all 230 space groups are present in the data
# Embedding doesn't support uint8. Sad!
dtype = torch.int64
cascade_order = ("elements", "symmetry_sites", "symmetry_sites_enumeration")
# (N_i, d_i, pad_i)
assert max_enumeration + 1 == enumeration_stop
assert max_enumeration + 2 == enumeration_pad
enumeration_mask = max_enumeration + 3
assert enumeration_mask < torch.iinfo(dtype).max

cascade = (
    (len(element_to_ids), 64, torch.tensor(element_to_ids[PAD_TOKEN], dtype=dtype, device=device)),
    (len(site_to_ids), 64 - 1, torch.tensor(site_to_ids[PAD_TOKEN], dtype=dtype, device=device)),
    (enumeration_mask + 1, None, torch.tensor(enumeration_pad, dtype=dtype, device=device))
)
model = CascadeTransformer(
    n_start=n_space_groups,
    cascade=cascade,
    n_head=1,
    d_hid=256,
    n_layers=4,
    dropout=0.1,
    use_mixer=True).to(device)
# Our dynamic discard of predicting PAD calls for frequent recompilation
# model = torch.compile(model)

In [5]:
import torch
pad_dict = {
    "elements": element_to_ids[PAD_TOKEN],
    "symmetry_sites": site_to_ids[PAD_TOKEN],
    "symmetry_sites_enumeration": enumeration_pad
}
mask_dict = {
    "elements": element_to_ids[MASK_TOKEN],
    "symmetry_sites": site_to_ids[MASK_TOKEN],
    "symmetry_sites_enumeration": enumeration_mask
}
trainer = WyckoffTrainer(
    model, torch_datasets, pad_dict, mask_dict, cascade_order, "spacegroup_number", max_len, device, dtype=dtype)
trainer.train(epochs=100000, val_period=20)

wandb: Currently logged in as: kazeev. Use `wandb login --relogin` to force relogin


wandb: Tracking run with wandb version 0.17.0


wandb: Run data is saved locally in /home/kna/WyckoffTransformer/wandb/run-20240524_035428-1zfmt5l0
wandb: Run `wandb offline` to turn off syncing.


wandb: Syncing run cerulean-cloud-236


wandb: ⭐️ View project at https://wandb.ai/kazeev/WyckoffTransformer


...

Epoch 38160 val_loss_epoch 16.88688087463379 saved to checkpoints/2024-05-24_03-54-27/best_model_params.pt


wandb:                                                                                


wandb: 
wandb: Run history:
wandb:             epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb: known_cascade_len █▅▁▁▁▅▁▅█▁█▅▅█▁█▁▅▅▅▅▁▁▅█▁█▁▅█▁▁▅▁▁▁██▁▁
wandb:     known_seq_len █▁▅▄▆▅▇▁▂▄▄▃▁█▄▇▁▄▅▃▇▅▆▅▆▅█▁▅▄▂██▅▇▃▄▆▂▅
wandb:                lr █▄▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:  train_loss_batch ▁█▁▁▁▁▁▂▁▁▁▁▂▁▁▁▆▁▁▁▁▁▁▁▁▁▁▆▁▁▅▁▁▁▁▁▁▁▃▁
wandb:  train_loss_epoch █▆▆▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:    val_loss_epoch █▆▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:             epoch 100000
wandb: known_cascade_len 0
wandb:     known_seq_len 1
wandb:                lr 0.0
wandb:  train_loss_batch 83489.85156
wandb:  train_loss_epoch 16.25737
wandb:    val_loss_epoch 16.88753
wandb: 


wandb: 🚀 View run cerulean-cloud-236 at: https://wandb.ai/kazeev/WyckoffTransformer/runs/1zfmt5l0
wandb: ⭐️ View project at: https://wandb.ai/kazeev/WyckoffTransformer
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 1 other file(s)


wandb: Find logs at: ./wandb/run-20240524_035428-1zfmt5l0/logs
